<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/logreg/logreg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression Example: Predicting Titanic Passenger Survival

In [1]:
from pyspark.sql import SparkSession

In [2]:
SparkSession = SparkSession.builder.appName('titanic').getOrCreate()

In [3]:
df = SparkSession.read.csv('titanic.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
my_cols = df.select(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])

In [6]:
my_final_data = my_cols.na.drop()

In [7]:
from pyspark.ml.feature import (VectorAssembler,
                                VectorIndexer, OneHotEncoder, StringIndexer)

In [8]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [9]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [10]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkVec'],
                           outputCol='features')

In [11]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [12]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [13]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder,
                            assembler, log_reg_titanic])

In [14]:
train_data, test_data = my_final_data.randomSplit([0.8,0.2])

In [15]:
fit_model = pipeline.fit(train_data)

In [16]:
results = fit_model.transform(test_data)

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [19]:
AUC = my_eval.evaluate(results)

In [20]:
AUC

0.7664473684210528